Tabular Data için Çalışan Koşullu GAN'ın Dinamik hale getirilmesi ve Fonksiyonelleştirilmesi

In [1]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


In [2]:
file_name = "20ft_12k.csv"
data_path = "C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/input_reals/{0}".format(file_name)
data = pd.read_csv(data_path, sep=";")

data.loc[data[data["CROSSING"] >= data["CROSSING"].median()].index, "y"] = 1
data.loc[data[data["CROSSING"] < data["CROSSING"].median()].index, "y"] = 0

df = data.copy()
df.head()

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL,...,SPRINT_SPEED,AGILITY,REACTION,BALANCE,SHOT_POWER,JUMPING,STAMINA,STRENGTH,LONG_SHOTS,y
0,56.0,28.0,54.0,68.0,55.0,58.0,58.0,49.0,65.0,62.0,...,56.0,65.0,56.0,66.0,58.0,55.0,65.0,55.0,61.0,0.0
1,21.0,20.0,26.0,27.0,21.0,30.0,8.0,11.0,75.0,34.0,...,56.0,60.0,75.0,56.0,26.0,72.0,72.0,71.0,20.0,0.0
2,49.0,67.0,47.0,73.0,59.0,71.0,63.0,62.0,67.0,72.0,...,72.0,75.0,69.0,72.0,67.0,68.0,81.0,58.0,62.0,0.0
3,41.0,62.0,60.0,60.0,52.0,70.0,49.0,38.0,48.0,68.0,...,69.0,74.0,59.0,71.0,59.0,62.0,57.0,60.0,51.0,0.0
4,60.0,41.0,46.0,70.0,60.0,62.0,47.0,31.0,68.0,67.0,...,65.0,72.0,68.0,71.0,42.0,72.0,75.0,63.0,47.0,0.0


In [3]:
# data_path = "C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/input_reals/10ft_3k.csv"
# data = pd.read_csv(data_path, sep=";")
# df = data.copy()

# # preprocess
# le = preprocessing.LabelEncoder()
# for i in ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']:
#     df[i] = le.fit_transform(df[i].astype(str))

scaler = StandardScaler()
condition_feature = "y"
X_train = scaler.fit_transform(df.drop(condition_feature, 1))
y_train = df[condition_feature].values

C:\Users\KALYBE~1\AppData\Local\Temp/ipykernel_8308/3217749644.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = scaler.fit_transform(df.drop(condition_feature, 1))


In [4]:
# config
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 32
latent_dim = 100
out_shape = X_train.shape[1]

In [5]:
# loss & optimizer
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [6]:
# koşullar:
list_condition = list(np.unique(y_train))

generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = {}

for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    print(train_data_cond.shape)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data_cond).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets[cond] = train_dataset
    
    discriminator = Discriminator(out_shape=out_shape)
    generator = Generator(out_shape=out_shape)
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns.drop(condition_feature))
    gen[condition_feature] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)

(5861, 20)
Epoch 1/50


C:\Users\kalybeai-dxlc693\anaconda3\envs\gpu_tensorflow\lib\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


184/184 [==============================] - 3s 6ms/step - d_loss: 0.6197 - g_loss: 0.7340
Epoch 2/50
184/184 [==============================] - 1s 5ms/step - d_loss: 0.6208 - g_loss: 0.8890
Epoch 3/50
184/184 [==============================] - 1s 6ms/step - d_loss: 0.5961 - g_loss: 1.0174
Epoch 4/50
184/184 [==============================] - 1s 6ms/step - d_loss: 0.5519 - g_loss: 1.1597
Epoch 5/50
184/184 [==============================] - 1s 6ms/step - d_loss: 0.5772 - g_loss: 1.1170
Epoch 6/50
184/184 [==============================] - 1s 6ms/step - d_loss: 0.5960 - g_loss: 1.0190
Epoch 7/50
184/184 [==============================] - 1s 6ms/step - d_loss: 0.5890 - g_loss: 0.9621
Epoch 8/50
184/184 [==============================] - 1s 5ms/step - d_loss: 0.5927 - g_loss: 0.9436
Epoch 9/50
184/184 [==============================] - 1s 5ms/step - d_loss: 0.5830 - g_loss: 0.9369
Epoch 10/50
184/184 [==============================] - 1s 5ms/step - d_loss: 0.5796 - g_loss: 0.9627
Epoch 11/5

192/192 [==============================] - 1s 6ms/step - d_loss: 0.4761 - g_loss: 1.2898
Epoch 33/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4680 - g_loss: 1.3115
Epoch 34/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4648 - g_loss: 1.2964
Epoch 35/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4669 - g_loss: 1.3213
Epoch 36/50
192/192 [==============================] - 1s 6ms/step - d_loss: 0.4620 - g_loss: 1.3319
Epoch 37/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4575 - g_loss: 1.3327
Epoch 38/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4540 - g_loss: 1.3479
Epoch 39/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4539 - g_loss: 1.3458
Epoch 40/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4538 - g_loss: 1.3570
Epoch 41/50
192/192 [==============================] - 1s 5ms/step - d_loss: 0.4486 - g_loss: 1.3659
Ep

In [7]:
# generated_df.to_csv("C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/output_synt/adult.csv",
#               index=False, sep=",")

In [8]:
gen_features = scaler.inverse_transform(generated_df.drop(condition_feature, 1))
gen_df = pd.DataFrame(gen_features, columns=data.columns.drop(condition_feature))
gen_df[condition_feature] = generated_df.loc[:, condition_feature].values
gen_df.to_csv("C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/output_synt/{0}".format(file_name),
              index=False, sep=",")

C:\Users\KALYBE~1\AppData\Local\Temp/ipykernel_8308/2010494955.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  gen_features = scaler.inverse_transform(generated_df.drop(condition_feature, 1))


In [9]:
gen_df

,CROSSING,FINISSHING,HEADING_ACCURACY,SHORT_PASSING,VOLLEYS,DRIBBLING,CURVE,FREE_KICK_ACCURACY,LONG_PASSING,BALL_CONTROL,...,SPRINT_SPEED,AGILITY,REACTION,BALANCE,SHOT_POWER,JUMPING,STAMINA,STRENGTH,LONG_SHOTS,y
0,57.369198,53.038166,67.785332,65.801987,62.717884,58.477013,49.674732,47.460831,60.358841,60.419006,...,59.147877,61.022701,64.978867,59.573414,67.071388,76.026398,78.978416,74.700813,57.496456,0.0
1,50.990524,61.410789,54.064697,62.089520,44.034294,54.034668,53.334137,38.564259,54.959576,58.579166,...,73.454659,54.058746,61.665184,53.327736,69.724762,57.965775,80.399536,80.388718,65.665611,0.0
2,56.857319,53.314262,70.696747,73.294975,64.709000,59.477791,49.803329,65.792801,65.506424,64.538033,...,67.192078,77.428825,62.577820,70.020142,74.606041,75.185661,81.847977,58.529724,69.007797,0.0
3,47.712204,69.368919,74.999847,58.860401,58.536587,57.842995,49.046814,37.339455,45.302471,69.514946,...,64.701317,61.186234,60.054466,70.494019,76.805496,72.774109,75.325188,74.307831,66.370018,0.0
4,38.932060,68.526047,63.752682,66.746307,50.448475,51.851318,43.077579,62.629627,45.311783,72.424850,...,59.835274,57.269634,65.995094,68.093445,77.841347,73.610924,76.804642,80.353645,67.554886,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,65.041573,54.693844,57.667202,68.633949,43.400078,75.780083,70.655518,51.999325,72.449669,74.508553,...,65.225441,70.613846,60.053474,68.865524,64.439827,67.473267,59.581757,75.317574,72.676033,1.0
11996,64.390335,61.917274,64.056923,67.948135,44.459702,70.798767,53.474060,68.557487,65.607994,70.640816,...,72.978882,62.563862,67.489647,57.104622,72.585083,59.688934,68.180122,76.652412,64.440277,1.0
11997,65.579224,29.390810,68.536766,67.846573,30.058270,56.990070,52.536694,39.267315,71.641106,57.124081,...,72.419411,53.638695,60.053474,55.020351,46.016037,57.898792,67.067268,68.711716,37.239124,1.0
11998,65.472084,56.768116,62.455875,69.033340,47.014202,66.850983,62.481159,58.368164,65.249588,68.740540,...,70.762413,71.989563,62.712570,69.828949,64.751091,66.275459,74.377098,66.075325,66.119301,1.0


In [10]:
dsafasafafdfdsfsdafsd

NameError: name 'dsafasafafdfdsfsdafsd' is not defined

Bu repo görsel üreten bir gan modelini ele alır (vanilla)
Bu repo üzerinde 2 aşamada conditional tabular data için veri üretiyor olacağız
* Koşullu yapma
* Tabular datada çalışıyor hale getirme

Şu anda 2. aşamada çalışılınıyor

In [ ]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path 

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

#### Prepare MNIST data

In [ ]:
data_path = Path("./gan/datasets/data")

In [ ]:
from keras.datasets import mnist

In [ ]:
(train_image, train_labels), (_, _) = mnist.load_data()

In [ ]:
train_images = train_image.reshape(train_image.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 32
latent_dim = 100

In [ ]:
# belirli bir koşul için veri seti hazırlama (koşul "= 5" olması)
indices = np.where(train_labels == 5)[0]
train_images_5 = []
for i in range(len(train_labels)):
    if i in indices:
        train_images_5.append(train_images[i])
        
train_images_5 = np.array(train_images_5)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images_5).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# toplamda 60.000 görsel var
train_images.shape

In [ ]:
# toplamda 5421 adet "5" görseli var
train_images_5.shape

Aşağıdaki model oluşturma adımında hangi koşul yazılırsa yazılsın model gerçek veya sahte ayrımı yapmaya çalışacaktır.
Bu yüzden önemli nokta veri setini ayrıştırıp ona vermektir örneğin sadece 5 için görsel çizdirmeye çalışalım.

#### Prepare Model

In [ ]:
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [ ]:
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)

In [ ]:
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)

In [ ]:
gan.fit(train_dataset, epochs=10)

#### Testing Generator

In [ ]:
noise = tf.random.normal([1, 100])

In [ ]:
generated_image = generator(noise)

In [ ]:
generator.summary()

In [ ]:
pred = discriminator(generated_image)

In [ ]:
pred

In [ ]:
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
# deney
# ilk deney koşula uygun n adet yani birden fazla sentetik veri üretebilir miyiz?
# ikinci deney farklı koşullar için tekrar ve tekrar model eğitmek ile tek bir model eğitmek arasında ne gibi maliyet farkı var?

In [ ]:
# deney 1:

In [ ]:
latent_dim = 100
num_images = 49
random_latent_vectors = tf.random.normal(shape=(num_images, latent_dim))
generated_images = generator(random_latent_vectors)

In [ ]:
plt.imshow(generated_images[10])

In [ ]:
# deney 2:

In [ ]:
%%time
# vanilla gan tüm veriler ile eğitiliyor.
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(train_dataset, epochs=50)

32 dk 46 saniye - 60000x28x28

In [ ]:
%%time
for spec in range(10):
    indices = np.where(train_labels == spec)[0]
    train_images_spec = []
    for i in range(len(train_labels)):
        if i in indices:
            train_images_spec.append(train_images[i])

    train_images_spec = np.array(train_images_spec)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images_spec).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    print("for spec:", spec)
    print("len train_images_spec:", len(train_images_spec))
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(train_dataset, epochs=50)

33 dk - her veri ayrı ayrı

In [ ]:
# Aşarısı sadece modellerin çıktısını test etmek için

In [ ]:
%%time
list_img = []
for spec in range(10):
    indices = np.where(train_labels == spec)[0]
    train_images_spec = []
    for i in range(len(train_labels)):
        if i in indices:
            train_images_spec.append(train_images[i])

    train_images_spec = np.array(train_images_spec)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images_spec).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    print("spec:", spec)
    print("len train_images_spec:", len(train_images_spec))
    print("ilk on:", indices[:10])
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(train_dataset, epochs=20)
    noise = tf.random.normal([1, 100])
    generated_image = generator(noise)
    list_img.append(generated_image[0, :, :, 0])
    #plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
for i in range(10):
    plt.imshow(list_img[i])
    plt.show()

Not: Şu anda ilk aşama tamamlandı. Yani görsel veriler için belirli bir koşula uygun veri üretimi vanilla gan kullanılarak gerçekleştirildi.

Birden fazla örnek oluşturma veya oluşturulan örneklerin gösterilmesi ile ilgili bir problem gözlemlenmedi.

Ayrıca modellerin tek tek eğitilmesi veya tüm verilerin aynı anda koşuldan bağımsız eğitilmesi konusunda bir zaman farkı bulunmamaktadır.

Bizim verilerimizin çok fazla boyuta sahip olduğu için bu kadar uzun sürdü eğitimler örneğin, adult veri seti 32.500x15 boyutundadır, bizim görsel veri setimiz 60.000x48x48 yani yaklaşık 96 katı boyutunda.

Burada bir sonraki aşama tabular data için eğitim gerçekleştirebilmektir.
Daha sonrasında işlemlerin fonksiyonlaştırılması ve nesneye yönelimli programlama yapısında py dosyası formatına getirilmesidir.

En sonunda da bu modelin eğitimi hızlandırılmaya çalışılabilir. Veriyi temsil eden en iyi bir örneklem seçilebilir vs. veya paralelleştirme denenebilir. Modelin içine koşul verilmeye çalışılabilir. Early stopping eklenebilir. Veri boyutuna bağlı epoch sayısı belirlenebilir. Fakat version 1 için elde çalışan bir yapının bulunması iyi olacaktır. 

Şu anda 2. aşamadayız yani görsel verisi için eğittiğimiz modeli tabular data için eğiteceğiz.

In [ ]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path 

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
data_path = "C:/Users/kalybeai-dxlc693/Desktop/GANS/adult.csv"

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
df = data.copy()

In [ ]:
le = preprocessing.LabelEncoder()
for i in ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']:
    df[i] = le.fit_transform(df[i].astype(str))

In [ ]:
df

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(df.drop('income', 1))
y_train = df['income'].values

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 32
latent_dim = 100

In [ ]:
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [ ]:
X_train = np.float32(X_train)
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
%%time
# vanilla gan tüm veriler ile eğitiliyor.
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)

In [ ]:
gan.fit(train_dataset, epochs=50)

In [ ]:
noise = tf.random.normal([1, 100])

In [ ]:
generated_data = generator(noise)

In [ ]:
np.array(generated_data)[0]

In [ ]:
# önce train içindeki kısım incelenecek
# sonra koşula uygun yapılmaya çalışılacaktır
# class sayısı dinamik yapılacak

In [ ]:
generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = []
list_condition = [0, 1]
for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets.append(train_dataset)
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
    gen[data.columns[-1]] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)
    

In [ ]:
generated_df

In [ ]:
generated_df.to_csv("C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/output_synt/adult.csv",
              index=False, sep=",")

In [ ]:
gen_features = scaler.inverse_transform(generated_df.drop('income', 1))

In [ ]:
gen_df = pd.DataFrame(gen_features, columns=data.columns[:-1])

In [ ]:
gen_df[data.columns[-1]] = generated_df.iloc[:, -1].values

In [ ]:
gen_df.to_csv("C:/Users/kalybeai-dxlc693/Desktop/GANS/modular-conditional-gan-main/datasets/output_synt/adult.csv",
              index=False, sep=",")

In [ ]:
conditional_datasets[0]
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(conditional_datasets[0], epochs=10)
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = cond

In [ ]:
conditional_datasets[0]
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(conditional_datasets[1], epochs=10)
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen1 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen1[data.columns[-1]] = cond

In [ ]:
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = cond

In [ ]:
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = 0

In [ ]:
data.columns[-1]

In [ ]:
data.columns[:-1]

Tabular Data için Çalışan Koşullu GAN'ın Dinamik hale getirilmesi ve Fonksiyonelleştirilmesi

In [ ]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

In [ ]:
data_path = "C:/Users/kalybeai-dxlc693/Desktop/GANS/adult.csv"
data = pd.read_csv(data_path)
df = data.copy()

# preprocess
le = preprocessing.LabelEncoder()
for i in ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']:
    df[i] = le.fit_transform(df[i].astype(str))

scaler = StandardScaler()
X_train = scaler.fit_transform(df.drop('income', 1))
y_train = df['income'].values

In [ ]:
# config
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 32
latent_dim = 100

In [ ]:
# loss & optimizer
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [ ]:
# koşullar:
list_condition = list(np.unique(y_train))
condition_feature = "income"

generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = {}

for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    print(train_data_cond.shape)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data_cond).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets[cond] = train_dataset
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns.drop(condition_feature))
    gen[condition_feature] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)

In [ ]:
# koşullar:
list_condition = [0, 1]

generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = {}


for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets[cond] = train_dataset
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
    gen[data.columns[-1]] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)
    

In [ ]:
# koşullar:
list_condition = list(np.unique(y_train))
condition_feature = "income"

generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = {}

for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    print(train_data_cond.shape)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_data_cond).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets[cond] = train_dataset
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns.drop(condition_feature))
    gen[condition_feature] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)

In [ ]:
generated_df["income"].value_counts()